In [1]:
import os, glob, sys
import re
import pandas as pd   #openpyxl must be installed additonaly

mymodules = os.fspath('../modules')
if mymodules not in sys.path:
    sys.path.append(mymodules)
    
import bruker as b
import general as g
import importlib
b = importlib.reload(b)
g = importlib.reload(g)

In [2]:
# scanpath = '/mnt/c/Users/felix/Documents/MRDATA/20190306_125108_ZK_SCI_3RD_SHAM_20190228_221_1_2'
# scanpath = '/mnt/c/Users/felix/Documents/MRDATA/20190116_141334_ZK_SCI_3RD_MILD_20190116_212_1_1'

# scanpath = '/mnt/c/Users/felix/Documents/MRDATA/20181127_083057_ZK_3RD_SC_SEV_20181127_196_1_1'
# scanpath = '/mnt/c/Users/felix/Documents/MRDATA/20190306_125108_ZK_SCI_3RD_SHAM_20190228_221_1_2'


def process_scanpath(scanpath, min_scantime=120, exclude_method='FieldMap', oneup=False):
    """
    to be run in single or batch mode:
    create new *_Analysis folder, with specified 2dseq inside converted as nii.gz
    To filter out tripilots or failed scans, adjust e.g. min_scantime or exclude_method (by default FieldMap)
    """
    if oneup:
        scanpath = os.path.dirname(scanpath)
    
    methodfiles = glob.glob(scanpath + '/**/method')
    analysispath = f'{scanpath}_Analysis'
    if not os.path.isdir(analysispath):
        os.mkdir(analysispath)

    df = pd.DataFrame()
    for number, line in enumerate(methodfiles):
        path = os.path.dirname(line)
        df.loc[number, 'path'] = path
        _, df.loc[number, 'scannumber'] = os.path.split(path)
        _, df.loc[number, 'ID'] = os.path.split(_)
        df.loc[number, 'Method'], df.loc[number, 'ScanTime'], df.loc[number, 'Orientation (ParaVision)'] = b.read_method_default(line)


    for index, row in df.iterrows():
        df.loc[index,'ScanTime_s'] = b.scantime_to_s(row['ScanTime'])
        df_filtered = df.query(f"(ScanTime_s > {str(min_scantime)}) and (Method != '{exclude_method}')")
        
    df_filtered.to_excel(os.path.join(analysispath,'scan_overview.xlsx'))    
    
    return df_filtered, analysispath

# df_filtered = process_scanpath(scanpath)


In [ ]:
# Convert all that are left in df_filtered

def convert_from_df(df_filtered, outpath):
    for index, row in df_filtered.iterrows():
            print(index, 'converting scan ', row['scannumber'], row['Method'])
            b.convert_2dseq(row['path'], outpath)

In [4]:
# run above for whole folder of animals

mainpath = '/mnt/d/DATA/All_Relaxation'

# good: it will skip irrelevant folders that don't contain a subject file which could crash other code
# bad: no check if already converted
all_samples = g.get_filepaths(mainpath, fileending='subject', subfolders=True)

for sample in all_samples:
    print(sample)
    df_filtered, outpath = process_scanpath(sample, oneup=True)
    convert_from_df(df_filtered, outpath)


/mnt/d/DATA/All_Relaxation/20181119_092406_ZK_3RD_SC_S_20181119_194_1_1/subject
1 converting scan  13 RAREVTR
2 converting scan  15 MSME
3 converting scan  18 RAREVTR
4 converting scan  19 MSME
6 converting scan  20 MSME
7 converting scan  21 MSME
8 converting scan  22 FLASH
9 converting scan  23 DtiEpi
10 converting scan  24 DtiEpi
12 converting scan  26 DtiEpi
15 converting scan  29 FISP
17 converting scan  31 FISP
18 converting scan  32 RAREVTR
19 converting scan  33 MSME
20 converting scan  34 RAREVTR
21 converting scan  35 MSME
22 converting scan  36 MSME
23 converting scan  37 MSME
24 converting scan  38 FLASH
25 converting scan  39 DtiEpi
26 converting scan  4 FISP
27 converting scan  40 DtiEpi
28 converting scan  5 FISP
/mnt/d/DATA/All_Relaxation/20181121_171153_ZK_SC_SEV_20181121_193_1_1/subject
1 converting scan  10 MSME
2 converting scan  11 FLASH
3 converting scan  12 DtiEpi
4 converting scan  13 DtiEpi
7 converting scan  3 FISP
8 converting scan  4 FISP
9 converting scan  

In [10]:
# File conversion, with selection criteria

#todo: add a check, if .nii.gz already exists, skip conversion
for index, row in df.iterrows():
    if row['Method'] != 'FieldMap' and row['scannumber'] == '12':
        print(index, 'converting scan ', row['scannumber'], row['Method'])
        b.convert_2dseq(row['path'], analysispath)

3 converting scan  12 MSME
